## UAV fleet Simulation 

This is for testing propouses

In [ ]:
import os
import subprocess
import numpy as np

# Custom modules
from vehicle_logic import VehicleLogic
from helpers.change_coordinates import heading_to_yaw,find_spawns
from simulators.gazebo.gazebo import update_world
from helpers.change_coordinates import local2global,heading_to_yaw,find_spawns
from helpers.visualization.plots import plot_3d_interactive
from simulators.QGroundControl.qgc import add_qgc_links,delete_all_qgc_links
from plan import Plan,State
from gcs import GCS

Kill all related process

In [2]:
delete_all_qgc_links()
for process in ["QGroundControl", "sim_vehicle.py", "arducopter", "gazebo", "mavproxy"]:
    os.system(f"pkill -9 -f {process}")

🧼 All QGroundControl links deleted (count=0).


Paths

In [3]:
QGC_path='~/QGroundControl.AppImage'
gazebo_word_path="gazebo_worlds/runway.world"
ardupilot_vehicle_path='~/ardupilot/Tools/autotest/sim_vehicle.py'

## Choose Simulator

In [4]:
simulator ='QGroundControl' # '' # 'gazebo' #

## Create Plans

In [5]:
offsets = [ #east, north, up, heading
    (5, 5, 0, 90), 
    (10, 0, 0, 45),
    (-5, -10, 0, 225),
    (-15, 0, 0, 0),
    (0, -20, 0, 0),
]

n_uavs = len(offsets)


local_paths=[Plan.create_square_path(side_len=dist,alt= 5) for dist in (5,7,10,12,15)]

homes=np.array([offset[:3] for offset in offsets])
gcs=GCS(positions=homes.copy())

global_paths=[local2global(np.array(path), home,pairwise=True) for path,home in zip(local_paths,homes)]

plans=[Plan.basic(wps=path,alt=5,wp_margin=0.5) for path in local_paths]


colors=['Green','Blue','Red','Orange','Yellow',]
markers = {f'uav_{i+1}': {'pos':global_paths[i],'color':colors[i]} for i in range(n_uavs)}


if simulator == 'QGroundControl':
    qgc_origin=(-35.3633245,149.1652241,0,0)
    spawns=find_spawns(qgc_origin, offsets)
    add_qgc_links(n=n_uavs)
    

plot_3d_interactive(markers,title='Simulation Markers',expand=[0.2,0.2,0.6],ground=-0.05)



Plan 'basic' created — no connection yet 🧩
Action 'Pre-Arm Check' created — no connection yet 🧩
Step 'Check disarmed' created — no connection yet 🧩
Step 'Check EKF' created — no connection yet 🧩
Step 'Check GPS' created — no connection yet 🧩
Step 'Check system' created — no connection yet 🧩
Action 'Set Mode: GUIDED' created — no connection yet 🧩
Step 'Switch to GUIDED' created — no connection yet 🧩
Action 'Arm' created — no connection yet 🧩
Step 'arm' created — no connection yet 🧩
Action 'takeoff' created — no connection yet 🧩
Step 'takeoff' created — no connection yet 🧩
Action 'fly' created — no connection yet 🧩
Step 'go to  -> (0, 0, 5)' created — no connection yet 🧩
Step 'go to  -> (0, 5, 5)' created — no connection yet 🧩
Step 'go to  -> (5, 5, 5)' created — no connection yet 🧩
Step 'go to  -> (5, 0, 5)' created — no connection yet 🧩
Step 'go to  -> (0, 0, 5)' created — no connection yet 🧩
Action 'Land' created — no connection yet 🧩
Step 'land' created — no connection yet 🧩
Plan 'ba

Launch Vehicles

In [6]:
for i in range(n_uavs):
    vehicle_cmd = f"python3 {ardupilot_vehicle_path} -v ArduCopter -I{i} --sysid {i+1} --no-rebuild"
    if simulator == 'QGroundControl':
        spawn=','.join(map(str, spawns[i]))
        vehicle_cmd += f" --custom-location={spawn}"
    elif simulator == 'gazebo':
        vehicle_cmd += " -f gazebo-iris"
    subprocess.Popen(["gnome-terminal", "--", "bash", "-c", f"{vehicle_cmd}; exec bash"])

Create UAVs

In [7]:
uavs=[]
for i,(plan,home) in enumerate(zip(plans,homes)): 
    uavs.append(VehicleLogic(sys_id=i+1,
                    home=home,
                    plan= plan))

Vehicle 1: Step 'Check disarmed' is now connected ✅🔗
Vehicle 1: Step 'Check EKF' is now connected ✅🔗
Vehicle 1: Step 'Check GPS' is now connected ✅🔗
Vehicle 1: Step 'Check system' is now connected ✅🔗
Vehicle 1: Action 'Pre-Arm Check' is now connected ✅🔗
Vehicle 1: Action 'Pre-Arm Check' is now connected ✅🔗
Vehicle 1: Step 'Switch to GUIDED' is now connected ✅🔗
Vehicle 1: Action 'Set Mode: GUIDED' is now connected ✅🔗
Vehicle 1: Action 'Set Mode: GUIDED' is now connected ✅🔗
Vehicle 1: Step 'arm' is now connected ✅🔗
Vehicle 1: Action 'Arm' is now connected ✅🔗
Vehicle 1: Action 'Arm' is now connected ✅🔗
Vehicle 1: Step 'takeoff' is now connected ✅🔗
Vehicle 1: Action 'takeoff' is now connected ✅🔗
Vehicle 1: Action 'takeoff' is now connected ✅🔗
Vehicle 1: Step 'go to  -> (0, 0, 5)' is now connected ✅🔗
Vehicle 1: Step 'go to  -> (0, 5, 5)' is now connected ✅🔗
Vehicle 1: Step 'go to  -> (5, 5, 5)' is now connected ✅🔗
Vehicle 1: Step 'go to  -> (5, 0, 5)' is now connected ✅🔗
Vehicle 1: Step 'go

Launch Simulator

In [8]:
if simulator == 'QGroundControl':
    sim_path = os.path.expanduser(QGC_path)
    sim_cmd =[sim_path]
elif simulator == 'gazebo':
    # Convert to Gazebo format (name, x, y, z, roll, pitch, yaw)
    drones = [(east, north, up, 0, 0, heading_to_yaw(heading)) for i, (east, north, up, heading) in enumerate(offsets)]
    world_path = os.path.expanduser(gazebo_word_path)
    updated_world_path = update_world(drones,markers,world_path)
    sim_cmd = ["gazebo", "--verbose", updated_world_path] 
else:
    sim_cmd = []
if sim_cmd:
    simulator_process = subprocess.Popen(
                        sim_cmd,
                        stdout=subprocess.DEVNULL,  # Suppress standard output
                        stderr=subprocess.DEVNULL,  # Suppress error output
                        shell=False  # Ensure safety when passing arguments
                        )

## Execute Plan

In [9]:

while gcs.incomplete_missions:
    to_remove = set()
    for id in gcs.incomplete_missions:
        uav = uavs[id-1]
        if uav.plan.state == State.DONE:
            to_remove.add(id)
        else:
            uav.plan.act()
    gcs.update_missions(to_remove)

Vehicle 1: ▶️ Starting Step: Check disarmed
Vehicle 2: ▶️ Starting Step: Check disarmed
Vehicle 3: ▶️ Starting Step: Check disarmed
Vehicle 4: ▶️ Starting Step: Check disarmed
Vehicle 5: ▶️ Starting Step: Check disarmed
Vehicle 1: ✅ Step: Check disarmed is done
Vehicle 2: ✅ Step: Check disarmed is done
Vehicle 3: ✅ Step: Check disarmed is done
Vehicle 4: ✅ Step: Check disarmed is done
Vehicle 1: ▶️ Starting Step: Check EKF
Vehicle 2: ▶️ Starting Step: Check EKF
Vehicle 3: ▶️ Starting Step: Check EKF
Vehicle 4: ▶️ Starting Step: Check EKF
Vehicle 5: ✅ Step: Check disarmed is done
Vehicle 5: ▶️ Starting Step: Check EKF
Vehicle 1: ✅ Step: Check EKF is done
Vehicle 1: ▶️ Starting Step: Check GPS
Vehicle 3: ✅ Step: Check EKF is done
Vehicle 3: ▶️ Starting Step: Check GPS
Vehicle 5: ✅ Step: Check EKF is done
Vehicle 5: ▶️ Starting Step: Check GPS
Vehicle 4: ✅ Step: Check EKF is done
Vehicle 4: ▶️ Starting Step: Check GPS
Vehicle 2: ✅ Step: Check EKF is done
Vehicle 2: ▶️ Starting Step: Check

## Kill all related process

In [10]:
for process in ["QGroundControl", "sim_vehicle.py", "arducopter", "gazebo", "mavproxy"]:
    os.system(f"pkill -9 -f {process}")